In [64]:
import numpy as np
import random as rdm
import math

In [200]:
sigmoid_prime(0.15)

0.24859900667754298

In [199]:
sigmoid(1)

0.7310585786300049

In [279]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(X):
    return np.maximum(0,X)

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

def relu_prime(X):
    return np.array([1 if x >= 0 else 0 for x in X])

def activation(name):
    if name =='sigmoid':
        return sigmoid,sigmoid_prime
    elif name=='relu':
        return relu,relu_prime
    else:
        raise Exception('activation function not in [sigmoid,relu]')
        
def mean_square_error(y_hat,y):
    return np.square(y_hat - y).mean(axis = 1)

def mean_square_error_der(y_hat,y):
    if len(y) > 1:
        return 2*(y_hat - y).mean(axis = 1)
    else:
        return 2*(y_hat - y)

def square_sum(X):
    return np.square(X).sum()

def loss_function(name):
    if name=='mean_square_error':
        return mean_square_error , mean_square_error_der
    else:
        raise Exception('loss function not in [mean_square_error]')

def penalization(name):
    if name == 'square_sum':
        return square_sum
    else:
        raise Exception('penalization function not in [square_sum]')

In [368]:
class NeuralNetwork:
    def __init__(self,layers_parameters , global_parameters):
        self.layers = {l:{'W' : 0.1*np.random.random(size = (layers[l]['output_dim'],layers[l]['input_dim'])),
                          'B' : 0.1*np.random.random(size = (layers[l]['output_dim'],1)),
                          'g' : activation(layers[l]['activation'])[0],
                          'g_prime': activation(layers[l]['activation'])[1],
                          'shape':{'W':(layers[l]['output_dim'],layers[l]['input_dim']),
                                   'B':(layers[l]['output_dim'],1),
                                   'g_prime':(layers[l]['output_dim'],1),
                                   'input':(layers[l]['input_dim'],1),
                                   'output':(layers[l]['output_dim'],1)}
                         } for l in layers}
        
        self.input_dim = global_parameters['input']
        self.output_dim = global_parameters['output']
        self.layers_dim = global_parameters['layers']
        self.batch_size = global_parameters['batch']
        self.lr = global_parameters['learning_rate']
        
        self.loss = loss_function(global_parameters['loss'])[0]
        self.loss_der = loss_function(global_parameters['loss'])[1]
        
    def predict(self,X):
        if len(X) != self.input_dim:
            raise Exception('input of size {} instead of {}'.format(len(X),self.input_dim))
        intermediary_step = self.forward(X)
        return intermediary_step[-1][2]
    
    def loss_calculation(self,x,y):
        y_hat = self.predict(X)
        return self.loss(y_hat,h)
    
    def forward(self,x):
        intermediary_step = []
        if len(x) != self.input_dim:
            raise Exception('input of size {} instead of {}'.format(len(x),self.input_dim))
        x = x.reshape((self.input_dim,1))
        for n in range(self.layers_dim):
            z = np.dot(self.layers[n]['W'] , x) + self.layers[n]['B']
            z = z.reshape(self.layers[n]['shape']['output'])
            h = self.layers[n]['g'](z).reshape(self.layers[n]['shape']['output'])
            x = x.reshape(self.layers[n]['shape']['input'])
            intermediary_step.append([x , z , h])
            x = np.array(h)
            
        return intermediary_step
    
    def backward(self , intermediary_step , y):
        if len(intermediary_step) != self.layers_dim:
            raise Exception('dimensions do not match')
        dJ_dh = self.loss_der(intermediary_step[-1][2] , y.reshape((2,1)))
        dJ_dh.reshape((self.output_dim , 1))
        for i in range(len(intermediary_step)):
            k = len(intermediary_step) - i - 1
            x , z , h = intermediary_step[k]
            n , m = self.layers[k]['shape']['output'] , self.layers[k]['shape']['input']
            if x.shape != m or z.shape != n or h.shape != n:
                x.reshape(m)
                z.reshape(n)
                h.reshape(n)
            dJ_dz = dJ_dh * self.layers[k]['g_prime'](z)
            if dJ_dz.shape != n:
                dJ_dz = dJ_dz.reshape(n)
            dJ_dw = np.dot(dJ_dz,x.T)
            dJ_db = dJ_dz
            dJ_dh = np.dot(self.layers[k]['W'].T,dJ_dz)
            self.layers[k]['W'] = self.layers[k]['W'] - self.lr * dJ_dw
            self.layers[k]['B'] = self.layers[k]['B'] - self.lr * dJ_db
            
        return
    
    def train(self , X , Y):
        if len(X) != len(Y):
            raise Exception('X and Y dimension do not match')
        
        for i in range(len(X)):
            print(i,end = '\r')
            x , y = X[i] , Y[i]
            intermediary_step = self.forward(x)
            self.backward(intermediary_step,np.array([y]))  
        return self.score(X,Y)
    
    def score(self,X,Y):
        mean_square = 0
        for i in range(len(X)):
            y = self.predict(X[i])
            mean_square += np.square(y - Y[i])
        return np.sqrt(mean_square.mean())/len(X)

In [369]:
layers = {
          0:{'input_dim' : 2,
             'output_dim': 3,
             'activation': 'sigmoid'},
          1:{'input_dim' : 3,
             'output_dim': 2,
             'activation': 'relu'}
         }

parameters = {'input': 2,
              'output' : 2,
              'layers': 2,
              'loss': 'mean_square_error',
              'penalization':'square_sum',
              'batch':1,
              'learning_rate':0.1}

nn = NeuralNetwork(layers , parameters)

In [370]:
# data = []
# for i in range(10000):
#     y = i%2
#     dist = np.random.normal(loc = 1 + i%2,scale = 0.2)
#     x0 = np.random.uniform(low = -np.sqrt(dist/2),high = np.sqrt(dist/2))
#     sign = -1 if np.random.random() > 0.5 else 1
#     x1 = sign*np.sqrt(dist - x0**2)
#     data.append([x0,x1,y])
# X = np.array(data)[:,:2]
# Y = np.array(data)[:,2]

In [384]:
data = []
for i in range(100000):
    x , y = np.random.random() , np.random.random()
    norme = np.square(x ** 2 + y ** 2)
    angle = math.atan(y/x)
    data.append([x,y,norme,angle])
X = np.array(data)[:,:2]
Y = np.array(data)[:,2:]

In [388]:
nn.score(X, Y)

0.0017984188139190441

In [386]:
nn.train(X[:75000],Y[:75000])

0.002075081946097534

In [387]:
nn.score(X[:75000],Y[:75000]),nn.score(X[75000:],Y[75000:])

(0.002075081946097534, 0.0036048963836632646)

In [278]:
np.maximum(0,X)

array([[0.       ],
       [0.5868795]])

In [274]:
np.array([max(0,x) for x in X])

array([[0.154593 ],
       [0.5868795]])

In [145]:
z = np.dot(nn.layers[0]['W'] , X[0].reshape(2,1)) + nn.layers[0]['B']
z = z.reshape(nn.layers[0]['shape']['output'])

In [130]:
type(X[0].reshape((2,1)))

numpy.ndarray

In [147]:
nn.layers[0]['W']

array([[array([0.13445301]), array([1.24396206])],
       [array([0.54997813]), array([1.73574006])],
       [array([-1.0244631]), array([0.7470272])]], dtype=object)

In [142]:
nn.layers[0]['g'](np.array([1,2]).reshape(2,1))

array([[0.73105858],
       [0.88079708]])